In [ ]:
import shutil
from pathlib import Path
from omegaconf import OmegaConf
from typing import Callable
import warnings

def _get_timestamp_dirs(path: Path, is_date_dir: bool, is_multirun_dir: bool):
    if is_date_dir:
        timestamp_dirs = path.iterdir()
    elif (not is_date_dir) and is_multirun_dir:
        timestamp_dirs = [path]
    else:
        raise NotImplementedError()
    return timestamp_dirs

def datasets_to_storage(
    path: Path, is_date_dir: bool, is_multirun_dir: bool, target_dir: Path, subdir_id_creator: Callable
):
    timestamp_dirs = _get_timestamp_dirs(path, is_date_dir, is_multirun_dir)

    for timestamp_dir in timestamp_dirs:
        if not timestamp_dir.is_dir():
            continue

        for run_dir in timestamp_dir.iterdir():
            if not run_dir.is_dir():
                continue

            cfg = OmegaConf.load(run_dir / ".hydra" / "config.yaml")
            target_subdir = target_dir / subdir_id_creator(cfg)
            target_subdir.mkdir(exist_ok=True)

            shutil.copy2(str(run_dir / "dataset_dict.json"), target_subdir)

            dirs = [d for d in run_dir.iterdir() if d.is_dir() and d.name != "runs"]  # filtering tensorflow logs
            for d in dirs:
                split_dir = target_subdir / d.name
                if split_dir.exists():
                    warnings.warn(f"{split_dir} already exists. Skipping.")
                    continue
                print(split_dir)
                shutil.copytree(str(d), str(split_dir))


def experiment_results_to_storage(
    path: Path, is_date_dir: bool, is_multirun_dir: bool, target_dir: Path, subdir_id_creator: Callable
):
    timestamp_dirs = _get_timestamp_dirs(path, is_date_dir, is_multirun_dir)

    for timestamp_dir in timestamp_dirs:
        if not timestamp_dir.is_dir():
            continue

        for run_dir in timestamp_dir.iterdir():
            if not run_dir.is_dir():
                continue

            cfg = OmegaConf.load(run_dir / ".hydra" / "config.yaml")
            target_subdir = target_dir / subdir_id_creator(cfg)
            target_subdir.mkdir(exist_ok=True)

            weights = run_dir / "model.safetensors"
            training_args = run_dir / "training_args.bin"
            model_cfg = run_dir / "config.json"
            hydra_cfg = run_dir / ".hydra" / "config.yaml"
            for f_to_be_copied in [weights, training_args, model_cfg, hydra_cfg]:
                print(target_subdir / f_to_be_copied.name)
                shutil.copy2(str(f_to_be_copied), str(target_subdir / f_to_be_copied.name))


def delete_unused_checkpoints(path: Path, is_date_dir: bool, is_multirun_dir: bool):
    timestamp_dirs = _get_timestamp_dirs(path, is_date_dir, is_multirun_dir)

    for timestamp_dir in timestamp_dirs:
        if not timestamp_dir.is_dir():
            continue

        for run_dir in timestamp_dir.iterdir():
            if not run_dir.is_dir():
                continue

            for dir_to_delete in run_dir.glob("checkpoint*"):
                print(f"Deleting {dir_to_delete}")
                for obj in dir_to_delete.iterdir():
                    obj.unlink()
                dir_to_delete.rmdir()

## Shortcut models

- copy best model to `experiments` directory
- delete unused checkpoints

### SST2

In [ ]:
path = Path("/root/similaritybench/multirun/2024-03-20")
is_date_dir = True
is_multirun_dir = True
target_dir = Path("/root/similaritybench/experiments/models/nlp/shortcut")


def shortcut_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    shortcut_rate = str(cfg.shortcut_rate).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_scrate{shortcut_rate}"

experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, shortcut_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)


New shortcut rates that take base rate of classes into account

In [ ]:
path = Path("/root/similaritybench/multirun/2024-04-02")
is_date_dir = True
is_multirun_dir = True
target_dir = Path("/root/similaritybench/experiments/models/nlp/shortcut")


def shortcut_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    shortcut_rate = str(cfg.shortcut_rate).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_scrate{shortcut_rate}"

experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, shortcut_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)


### MNLI

In [ ]:
is_date_dir = False
is_multirun_dir = True
target_dir = Path("/root/similaritybench/experiments/models/nlp/shortcut")


def shortcut_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    shortcut_rate = str(cfg.shortcut_rate).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_scrate{shortcut_rate}"


for multirun_dirname in ["11-07-38", "11-07-39", "11-07-40", "11-07-41", "11-15-18"]:
    path = Path("/root") / multirun_dirname
    experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, shortcut_subdir_id)
    delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)

## Augmented models

### SST2
(copied over from Viserion)

In [ ]:
path = Path("/root/multirun/2024-03-20")
is_date_dir = True
is_multirun_dir = True
target_dir = Path("/root/similaritybench/experiments/models/nlp/augmentation")
datasets_target_dir = Path("/root/similaritybench/experiments/datasets/nlp/robustness")


def augmentation_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    augmentation_id = "eda"
    strength = str(float(cfg.augmentation.recipe.pct_words_to_swap)).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_{augmentation_id}_strength{strength}"


def augmentation_dataset_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    augmentation_id = "eda"
    strength = str(float(cfg.augmentation.recipe.pct_words_to_swap)).replace(".", "")
    return f"{dataset_id}_{augmentation_id}_strength{strength}"


# experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, augmentation_subdir_id)
# delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, augmentation_dataset_subdir_id)

### MNLI
(Some) MNLI models

In [ ]:

path = Path("/root/similaritybench/multirun/2024-04-03/11-19-13")
is_date_dir = False
is_multirun_dir = True
target_dir = Path("/root/similaritybench/experiments/models/nlp/augmentation")
datasets_target_dir = Path("/root/similaritybench/experiments/datasets/nlp/robustness")


def augmentation_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    augmentation_id = "eda"
    strength = Path(cfg.dataset.local_path).name.replace("mnli_eda_strength", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_{augmentation_id}_strength{strength}"


def augmentation_dataset_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    augmentation_id = "eda"
    strength = Path(cfg.dataset.local_path).name.replace("mnli_eda_strength", "")
    return f"{dataset_id}_{augmentation_id}_strength{strength}"


# experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, augmentation_subdir_id)
# delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)

path = Path("/root/similaritybench/multirun/2024-04-26")
is_date_dir = True
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, augmentation_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)

## Memorizing Models

### SST2

In [ ]:
is_date_dir = True
is_multirun_dir = True
target_dir = Path("/root/similaritybench/experiments/models/nlp/memorizing")
datasets_target_dir = Path("/root/similaritybench/experiments/datasets/nlp/memorizing")


def memorizing_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    strength = str(float(cfg.memorization_rate)).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_labels{cfg.memorization_n_new_labels}_strength{strength}"


def memorizing_dataset_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    strength = str(float(cfg.memorization_rate)).replace(".", "")
    return f"{dataset_id}_labels{cfg.memorization_n_new_labels}_strength{strength}"


path = Path("/root/similaritybench/multirun/2024-03-25")
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-03-26")
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

With updated memorization

In [ ]:
target_dir = Path("/root/similaritybench/experiments/models/nlp/memorizing")
datasets_target_dir = Path("/root/similaritybench/experiments/datasets/nlp/memorizing")


def memorizing_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    strength = str(float(cfg.memorization_rate)).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_labels{cfg.memorization_n_new_labels}_strength{strength}"


def memorizing_dataset_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    strength = str(float(cfg.memorization_rate)).replace(".", "")
    return f"{dataset_id}_labels{cfg.memorization_n_new_labels}_strength{strength}"


path = Path("/root/similaritybench/multirun/2024-04-03/17-09-20")
is_date_dir = False
is_multirun_dir = True
# experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
# delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-04-03/21-29-25")
is_date_dir = False
is_multirun_dir = True
# experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
# delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
# datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-04-04")
is_date_dir = True
is_multirun_dir = True
# experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
# delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
# datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

### MNLI

In [ ]:
target_dir = Path("/root/similaritybench/experiments/models/nlp/memorizing")
datasets_target_dir = Path("/root/similaritybench/experiments/datasets/nlp/memorizing")


def memorizing_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    strength = str(float(cfg.memorization_rate)).replace(".", "")
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}_labels{cfg.memorization_n_new_labels}_strength{strength}"


def memorizing_dataset_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    strength = str(float(cfg.memorization_rate)).replace(".", "")
    return f"{dataset_id}_labels{cfg.memorization_n_new_labels}_strength{strength}"


# path = Path("/root/11-16-46")
# is_date_dir = False
# is_multirun_dir = True
# experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
# delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
# datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-04-22/15-02-33")
is_date_dir = False
is_multirun_dir = True
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-04-22/15-02-34")
is_date_dir = False
is_multirun_dir = True
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-04-22/15-02-35")
is_date_dir = False
is_multirun_dir = True
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

path = Path("/root/similaritybench/multirun/2024-04-22/15-02-36")
is_date_dir = False
is_multirun_dir = True
experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, memorizing_subdir_id)
delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
datasets_to_storage(path, is_date_dir, is_multirun_dir, datasets_target_dir, memorizing_dataset_subdir_id)

## Standard Models

### SST2
Models are already copied over, just need to add the hydra configs.

In [ ]:
path = Path("/root/similaritybench/experiments/models/nlp/standard")
for model_path in path.iterdir():
    with (model_path / "path.txt").open() as f:
        original_hydra_path = f.readline()

    config_path = Path(original_hydra_path) / ".hydra" / "config.yaml"
    print(config_path, model_path / config_path.name)
    shutil.copy2(str(config_path), str(model_path / config_path.name))


### MNLI

In [ ]:
target_dir = Path("/root/similaritybench/experiments/models/nlp/standard")

def standard_subdir_id(cfg):
    dataset_id = cfg.dataset.path if not cfg.dataset.name else f"{cfg.dataset.path}__{cfg.dataset.name}"
    return f"{dataset_id}_pre{cfg.model.seed}_ft{cfg.dataset.finetuning.trainer.args.seed}"

# for time in ["15-02-18", "15-02-19", "15-02-20", "15-02-21", "15-02-22"]:
    # path = Path(f"/root/similaritybench/multirun/2024-04-22/{time}")
for time in ["22-18-24", "22-20-11", "22-20-56", "22-21-24", "22-22-03"]:
    path = Path(f"/root/similaritybench/multirun/2024-06-01/{time}")
    is_date_dir = False
    is_multirun_dir = True
    experiment_results_to_storage(path, is_date_dir, is_multirun_dir, target_dir, standard_subdir_id)
    delete_unused_checkpoints(path, is_date_dir, is_multirun_dir)
